In [3]:
import IPython.display as ipd
# % pylab inline
import os
import pandas as pd
import librosa
import glob 
import librosa.display
import random

from sklearn.preprocessing import StandardScaler
import numpy as np
import os

from tensorflow.keras.models import load_model

In [4]:
model=load_model("D:\\Voice Classification\\Code\\Gender_Classifier\\trained_models\\Gender_Classifier_NN.h5")

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 193)               37442     
_________________________________________________________________
dropout (Dropout)            (None, 193)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               24832     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2

In [28]:
# Again, the modified function to extract features from our new test data

def extract_features(files):
    
    # Sets the name to be the path to where the file is in my computer
    #file_name = os.path.join(os.path.abspath('new_test_male_female')+'/'+str(files.file))
    file_name="D:\\ML_Python\\voice classification\\"+files.file+".wav"

    # Loads the audio file as a floating point time series and assigns the default sample rate
    # Sample rate is set to 22050 by default
    X, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 

    # Generate Mel-frequency cepstral coefficients (MFCCs) from a time series 
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)

    # Generates a Short-time Fourier transform (STFT) to use in the chroma_stft
    stft = np.abs(librosa.stft(X))

    # Computes a chromagram from a waveform or power spectrogram.
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)

    # Computes a mel-scaled spectrogram.
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)

    # Computes spectral contrast
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)

    # Computes the tonal centroid features (tonnetz)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X),
    sr=sample_rate).T,axis=0)
        
    
    # We add also the classes of each file as a label at the end
    #label = files.label

    return mfccs, chroma, mel, contrast, tonnetz

In [29]:
files=pd.DataFrame({'file':["hello2"]})

In [30]:
features=files.apply(extract_features,axis=1)

In [31]:
features

0    ([-367.3588881757326, 75.43637931876452, -13.0...
dtype: object

In [32]:
x=np.array(features)

In [33]:
xf=np.load('D:\\Voice Classification\\Code\\Gender_Classifier\\features_label_test_new.npy', allow_pickle=True)

In [34]:
xf.shape

(2618,)

In [35]:
x.shape

(1,)

In [36]:
len(x[0])

5

In [37]:
features_new_test = []
for i in range(0, len(x)):
    features_new_test.append(np.concatenate((x[i][0], x[i][1], 
                x[i][2], x[i][3],
                x[i][4]), axis=0))

In [38]:
features_new_test

[array([-3.67358888e+02,  7.54363793e+01, -1.30827384e+01,  4.60420585e+00,
        -1.06760274e+00, -3.02601526e+00, -6.76953761e+00, -2.82676406e+00,
        -7.41565510e+00, -3.50610016e+00, -9.32932847e+00, -1.00899810e+01,
        -1.29823846e+01, -4.10814196e+00, -8.03616183e+00, -8.30792426e+00,
        -7.55726619e+00, -6.22920192e+00, -7.04699564e+00, -6.63087369e+00,
        -7.91987583e+00, -7.89171694e+00, -5.80421672e+00, -4.61875863e+00,
        -5.93380615e+00, -4.48403541e+00, -2.05443831e+00, -6.73115129e-01,
        -3.24645539e+00, -2.96307109e+00, -2.53403763e+00, -2.31011724e+00,
        -9.92799093e-01, -1.99066887e+00, -1.95602503e+00, -1.93234881e-02,
         1.17138320e+00, -8.54081116e-01, -1.01711893e+00,  1.04741163e+00,
         7.69746700e-01,  7.54250815e-01,  7.50907166e-01,  7.27749661e-01,
         7.29184344e-01,  7.43263188e-01,  8.00900542e-01,  7.99671246e-01,
         8.54201773e-01,  8.27824471e-01,  8.17004741e-01,  8.10402616e-01,
         8.2

In [39]:
X_new_test = np.array(features_new_test)

In [40]:
# load the scaler
from pickle import load
ss = load(open('D:\\Voice Classification\\Code\\Gender_Classifier\\scaler.pkl', 'rb'))

In [41]:
#ss=StandardScaler()
X_new_test_scaled = ss.transform(X_new_test)

In [42]:
X_new_test_scaled

array([[-6.45820241e-01, -2.19793907e+00,  6.61313146e-01,
        -2.65591491e+00,  5.26319274e-01, -9.90673163e-01,
         1.00355482e-01,  2.93976571e-01, -6.35038628e-01,
         2.77252560e-01, -1.84329987e+00, -5.87919243e-01,
        -2.90393697e+00, -1.54360324e-01, -2.14662484e+00,
        -1.14937082e+00, -1.21023035e+00, -1.69164377e+00,
        -7.72050662e-01, -1.78242303e+00, -1.29083750e+00,
        -2.11730209e+00, -1.09182854e+00, -1.01005780e+00,
        -1.76986743e+00, -1.11698757e+00, -6.44332620e-01,
         1.18829277e-01, -1.18406033e+00, -8.35465779e-01,
        -9.21792066e-01, -8.32325317e-01, -5.36238863e-01,
        -8.25399497e-01, -7.88152876e-01, -3.25542855e-01,
         2.08904362e-01, -5.60070692e-01, -5.07489115e-01,
         1.63645786e-01,  2.71649993e+00,  2.46200266e+00,
         2.30115517e+00,  1.91618058e+00,  1.91860898e+00,
         2.00058519e+00,  2.75077921e+00,  2.62204803e+00,
         3.36760313e+00,  3.11756099e+00,  3.18539394e+0

In [43]:
preds_new_test = model.predict_classes(X_new_test_scaled)

In [45]:
preds_new_test[0]

1

In [54]:
xf[0]

(array([-3.43151894e+02,  1.30807666e+02, -2.54582401e+01,  3.93869360e+01,
         5.78739313e+00,  8.59167441e+00, -2.30826018e+00, -6.26370499e+00,
        -1.49265264e+00, -2.64564332e+00,  2.13327045e+00, -2.67472739e+00,
         4.38225260e+00, -5.52471688e+00,  2.84004537e+00,  2.54154742e+00,
        -8.36582459e+00,  2.88208160e+00,  3.03161239e-01, -3.35218205e+00,
        -1.38677099e+00, -6.69648788e-01, -2.46809724e+00, -3.16088108e+00,
        -1.70967880e+00, -8.55692205e-01, -1.21527078e+00, -2.67280067e+00,
        -4.09102603e-01, -1.33746882e+00, -1.49478028e+00, -8.19055466e-01,
        -2.42719603e+00, -8.51065873e-01, -1.49274159e+00, -1.11107714e+00,
         4.22248251e-01, -1.61720483e+00, -1.12912938e+00, -1.23440105e+00]),
 array([0.62768872, 0.62923234, 0.65734485, 0.67268059, 0.66910285,
        0.66745062, 0.66377279, 0.66229756, 0.65492974, 0.63531823,
        0.64663837, 0.65517235]),
 array([1.32126304e-01, 1.84647980e-01, 1.00573468e+00, 2.54477662e+

In [125]:
from sklearn.preprocessing import MinMaxScaler

In [126]:
s=MinMaxScaler()